In [13]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Conv1D, MaxPooling1D, Flatten, Dropout
from keras.optimizers import Adam
from keras import backend as K
print(K.tensorflow_backend._get_available_gpus())
from IPython.display import display, clear_output
from scipy.special import softmax
import random
from sklearn import preprocessing
import matplotlib.pyplot as plt
import time

import os

['/job:localhost/replica:0/task:0/device:GPU:0']


In [10]:
# Deep Q-learning Agent
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen = 2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1  # exploration rate
        self.epsilon_min = 0.15
        self.epsilon_decay = 0.9995
        self.model = self._build_model()
        self.walk = deque(maxlen = 900)
        
    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        
        model.add(Dense(100, input_dim=self.state_size, activation='relu'))
        model.add(Dropout(0.2))
        model.add(BatchNormalization())
        
        model.add(Dense(100, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.2))
        
        
        
        model.add(Dense(100, activation='relu'))
        model.add(BatchNormalization())
        
        model.add(Dense(self.action_size, activation='softmax'))
        
        model.compile(loss='categorical_crossentropy', optimizer=Adam())
        model.summary()
        return model
    
    def remember(self, state, action, reward, next_state, done):
#         print(state)
#         print(reward)
        self.memory.append((state, action, reward, next_state, done))
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            chose = np.random.randint(0,4)
            return chose
        
        act_values = 0
        act_values = self.model.predict(state)
        chose = np.argmax(act_values)
#         print(act_values, chose)
        if chose == 0:
            return chose
        if chose == 1:
            return chose
        if chose == 2:
            return chose
        if chose == 3:
            return chose
        return act_values
    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        xs = []
        ys = []
        
        for state, action, reward, next_state, done in minibatch:
            target = reward

            if not done:
#                 if reward >= 0:
                
                target = reward + np.multiply (self.gamma , self.model.predict(next_state)[0] )
#                 else:
#                     target = np.multiply (-1 , self.model.predict(next_state)[0] )
            else:
                target = np.multiply (self.gamma , self.model.predict(next_state)[0] )
#             print(target)

            xs.append(state[0])

            ys.append(target)
#             ys = np.array(ys)
#             print(ys.shape)
        xs = np.array(xs)
        ys = np.array(ys)
        self.model.fit(xs, ys, epochs= 1, verbose=0 , batch_size=batch_size)
                
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
        random.shuffle(self.memory)
#         self.memory = deque(self.memory)
        
        
        if np.random.rand() > 0.8:
            self.memory.pop()

    
    def save_model(self):
        self.model.save('./checkpoint.h5')
        np.save("game_memory", self.memory)

    def load_model(self):
        self.model.load_weights('./checkpoint.h5')
        self.memory = np.load("game_memory.npy", allow_pickle=True)
        self.memory = deque(self.memory)

In [11]:
def count_consec_300(x):
    cnt = 0
    for i in range(len(x)):
        if x[i] >= 300:
            cnt += 1
        else:
            cnt = 0
    return cnt

def reward_greater(rew, no_improve):
    if rew < 0:
        no_improve += 1
    else:
        no_improve = 0
    return no_improve

In [14]:
env = gym.make('LunarLander-v2')
state_size = 8
action_size = 4
agent = DQNAgent(state_size, action_size)
# agent.load_model()
done = False
batch_size = 32
game_history = [0]

# game_history = np.load("game_history.npy")
explore = True
t_steps = 0
for e in range(10000):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
#     print(state.shape)
    total_reward = 0
    prev_reward = 0
    start = time.time()
    for timee in range(450):
        # env.render()
#         if time % 5 == 0:
#             print(time, end=', ')
        

        
        action = agent.act(state)
        
        next_state, reward, done, _ = env.step(action)
#         print(action, end='')
#         if done :
#             done = True
#             reward = -100
        

        next_state = np.reshape(next_state, [1, state_size])

        agent.remember(state, action, reward, next_state, done)
        state = next_state
        
        if done:
            break
        
        prev_reward = reward
        total_reward += reward
        if len(agent.memory) > 1000:
            agent.replay(batch_size)
            explore = False
#         print("diff:", reward - prev_reward,"prev:", prev_reward, " current:", reward, "total reward:", total_reward)


    game_history.append(total_reward)
#     if explore == False:
    print("round:{} score: {} mean:{} len of mem:{} spent:{}".format(e, total_reward, np.mean(game_history), len(agent.memory), time.time() - start))
        
    if e % 10 == 4:
        agent.save_model()
        np.save("game_history", game_history)

    done = False

        
np.save("positive_scene",agent.memory)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 100)               900       
_________________________________________________________________
dropout_5 (Dropout)          (None, 100)               0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 100)               400       
_________________________________________________________________
dense_10 (Dense)             (None, 100)               10100     
_________________________________________________________________
batch_normalization_8 (Batch (None, 100)               400       
_________________________________________________________________
dropout_6 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 100)               10100     
__________

round:64 score: -1401.9226415438673 mean:-320.382437522846 len of mem:2000 spent:9.805177211761475
round:65 score: -773.7531139372499 mean:-327.14916403649386 len of mem:2000 spent:7.098021507263184
round:66 score: -433.7188729192025 mean:-328.71636563771017 len of mem:2000 spent:3.2035703659057617
round:67 score: -1017.528446044532 mean:-338.6991494117221 len of mem:2000 spent:7.3602495193481445
round:68 score: -567.2580488456108 mean:-341.96427654649193 len of mem:2000 spent:6.085119724273682
round:69 score: -464.14967171637477 mean:-343.68519760522264 len of mem:2000 spent:4.826169490814209
round:70 score: -376.5187334080079 mean:-344.1412189358169 len of mem:2000 spent:4.233399391174316
round:71 score: -391.1215332876736 mean:-344.7847848858423 len of mem:2000 spent:3.5214953422546387
round:72 score: -443.19287075644 mean:-346.11462388409365 len of mem:2000 spent:2.7572073936462402
round:73 score: -1972.042085441539 mean:-367.7936567048596 len of mem:2000 spent:12.619820833206177
r

round:146 score: -977.2921198561788 mean:-500.9715793368997 len of mem:2000 spent:10.22830080986023
round:147 score: -1312.6901845822758 mean:-506.4193552110298 len of mem:2000 spent:9.172417879104614
round:148 score: -274.8997460188631 mean:-504.87589114974867 len of mem:2000 spent:3.6550567150115967
round:149 score: -829.6118788904614 mean:-507.02646060498523 len of mem:2000 spent:6.786843776702881
round:150 score: -398.13636202409504 mean:-506.3100783774794 len of mem:2000 spent:6.630048036575317
round:151 score: -694.9132634217635 mean:-507.5427789333243 len of mem:2000 spent:6.056920051574707
round:152 score: -490.2902523212167 mean:-507.4307495397393 len of mem:2000 spent:4.002099990844727
round:153 score: -392.0087941650861 mean:-506.6860917631286 len of mem:2000 spent:3.711543083190918
round:154 score: -469.4830912814146 mean:-506.4476109908099 len of mem:2000 spent:3.9236085414886475
round:155 score: -334.0165557888094 mean:-505.34932401500095 len of mem:2000 spent:4.110081911

round:228 score: -346.82755067560487 mean:-524.577019424637 len of mem:2000 spent:3.6058568954467773
round:229 score: -623.9425014671688 mean:-525.0071730265528 len of mem:2000 spent:4.9218833446502686
round:230 score: -425.0000524200089 mean:-524.5761078515245 len of mem:2000 spent:3.731381416320801
round:231 score: -362.24462227151633 mean:-523.8794061966747 len of mem:2000 spent:3.770700454711914
round:232 score: -735.1205242525289 mean:-524.7821460174263 len of mem:2000 spent:5.436336994171143
round:233 score: -433.20712649852584 mean:-524.3924650833033 len of mem:2000 spent:3.6005494594573975
round:234 score: -296.89255651821975 mean:-523.4284824198919 len of mem:2000 spent:3.646378517150879
round:235 score: -714.0679838355572 mean:-524.2328685018146 len of mem:2000 spent:9.115435123443604
round:236 score: -1105.1583038099816 mean:-526.6737316753782 len of mem:2000 spent:8.080748558044434
round:237 score: -1399.5623397930415 mean:-530.3259852658288 len of mem:2000 spent:10.8020889

KeyboardInterrupt: 

324